In [1]:
import os
import random
import pandas as pd
import yaml
yaml.Dumper.ignore_aliases = lambda *args : True
import itertools
pd.set_option('display.max_colwidth', None)

In [2]:
random.seed(1)

In [3]:
models =['Input', 'DCUNet', 'DPTNet', 'RegressionFCNN', 'SMoLnet', 'WaveUNet']
snrs = ['0', '-5', '-10', '-15', '-20', '-25', '-30']
df_all_snr_dict = {
    '0': '', 
    '-5': '',
    '-10': '',
    '-15': '',
    '-20': '',
    '-25': '',
    '-30' : ''
}

In [4]:
meta_data_dir = 'meta_data_for_yaml'
for snr in snrs:
    df_all_snr_dict[str(snr)] = pd.read_csv(f'meta_data_for_yaml/website_meta_data_snr_{snr}dB.csv')

In [5]:
df_all_snr_dict['-5']

,Unnamed: 0,Input,DCUNet,DPTNet,RegressionFCNN,SMoLnet,WaveUNet
0,0,configs/resources/Drone_Noise_Test_Data//drone_noise_out//Input/SX139.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DCUNet/-5db/data/DCUNet_SX139.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DPTNet/-5db/data/DPTNet_SX139.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//RegressionFCNN/-5db/data/RegressionFCNN_SX139.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//SMoLnet/-5db/data/SMoLnet_SX139.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//WaveUNet/-5db/data/WaveUNet_SX139.WAV.n121.wav
1,1,configs/resources/Drone_Noise_Test_Data//drone_noise_out//Input/SX139.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DCUNet/-5db/data/DCUNet_SX139.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DPTNet/-5db/data/DPTNet_SX139.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//RegressionFCNN/-5db/data/RegressionFCNN_SX139.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//SMoLnet/-5db/data/SMoLnet_SX139.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//WaveUNet/-5db/data/WaveUNet_SX139.WAV.n122.wav
2,2,configs/resources/Drone_Noise_Test_Data//drone_noise_out//Input/SI1129.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DCUNet/-5db/data/DCUNet_SI1129.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DPTNet/-5db/data/DPTNet_SI1129.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//RegressionFCNN/-5db/data/RegressionFCNN_SI1129.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//SMoLnet/-5db/data/SMoLnet_SI1129.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//WaveUNet/-5db/data/WaveUNet_SI1129.WAV.n121.wav
3,3,configs/resources/Drone_Noise_Test_Data//drone_noise_out//Input/SI1129.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DCUNet/-5db/data/DCUNet_SI1129.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DPTNet/-5db/data/DPTNet_SI1129.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//RegressionFCNN/-5db/data/RegressionFCNN_SI1129.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//SMoLnet/-5db/data/SMoLnet_SI1129.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//WaveUNet/-5db/data/WaveUNet_SI1129.WAV.n122.wav
4,4,configs/resources/Drone_Noise_Test_Data//drone_noise_out//Input/SX319.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DCUNet/-5db/data/DCUNet_SX319.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DPTNet/-5db/data/DPTNet_SX319.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//RegressionFCNN/-5db/data/RegressionFCNN_SX319.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//SMoLnet/-5db/data/SMoLnet_SX319.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//WaveUNet/-5db/data/WaveUNet_SX319.WAV.n122.wav
5,5,configs/resources/Drone_Noise_Test_Data//drone_noise_out//Input/SX319.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DCUNet/-5db/data/DCUNet_SX319.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DPTNet/-5db/data/DPTNet_SX319.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//RegressionFCNN/-5db/data/RegressionFCNN_SX319.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//SMoLnet/-5db/data/SMoLnet_SX319.WAV.n121.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//WaveUNet/-5db/data/WaveUNet_SX319.WAV.n121.wav
6,6,configs/resources/Drone_Noise_Test_Data//drone_noise_out//Input/SX409.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise_out//DCUNet/-5db/data/DCUNet_SX409.WAV.n122.wav,configs/resources/Drone_Noise_Test_Data//drone_noise

In [6]:
#get big dataframe
df = pd.concat([df_all_snr_dict['0'], df_all_snr_dict['-5'], df_all_snr_dict['-10'], df_all_snr_dict['-15'], df_all_snr_dict['-20'], df_all_snr_dict['-25'], df_all_snr_dict['-30']], ignore_index=True)

In [7]:
def get_combinations_of_index(df, idx):
    #all wavs except for the input to create combinations
    #Input will be our reference
    denoised_list = list(df.iloc[idx])[2:]
    #create pairs of twos
    #e.g. - (WaveUNet_SX139.WAV.n121.wav, RegressionFCNN_SX139.WAV.n121.wav) ...
    combinations = list(itertools.combinations(denoised_list, 2))

    # Final combination list with input appended to all 
    #e.g. - (input.wav, WaveUNet_SX139.WAV.n121.wav, RegressionFCNN_SX139.WAV.n121.wav) ...
    final_combination_list = []
    for c in combinations:
        #trick to append to tuple 
        c = (df['Input'][0], *c)
        final_combination_list.append(c)
    return final_combination_list    
    

In [8]:
final_list = []

In [9]:
for i in range(len(df)):
    res = get_combinations_of_index(df, i)
    for r in res:
        final_list.append(r)

In [10]:
#shuffle the list in place
random.shuffle(final_list)

In [11]:
def make_yaml_file(combinations, idx_meta='0_20'):
    print(f'''
    {len(combinations)} pages will be created for subjective evalutation
    ''')
    
    final_dict_template = {
    'testname': 'AB Test',
     'testId': 'ab_noloop',
     'bufferSize': 2048,
     'stopOnErrors': True,
     'showButtonPreviousPage': True,
     'remoteService': 'service/write.php',
     'pages': []
    }
    #final page which sends results
    final_page_template =  {'type': 'finish',
       'name': 'Thank you',
       'content': 'Thank you for attending',
       'showResults': False,
       'writeResults': True
    }
    pages=[]
    for i in range(len(combinations)):
        pages_dict_template = {'type': 'paired_comparison',
         'id': 'trialAB2',
         'name': None,
         'unforced': None,
         'content': 'test description AB',
         'showWaveform': True,
         'enableLooping': False,
         'reference': '',
         'stimuli': {'C1': '',
          'C2': ''}}
        pages.append(pages_dict_template)
        pages[i]['reference'] = combinations[i][0]
        pages[i]['stimuli']['C1'] = combinations[i][1]
        pages[i]['stimuli']['C2'] = combinations[i][2]
    
    pages.append(final_page_template)
    final_dict_template['pages'] = pages
    
    # dict to yaml
    yam_file_path = f'/Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/idx{idx_meta}.yaml'
    with open(yam_file_path, 'w') as f:
        yaml.dump(final_dict_template, f)
    print('Created yaml file at: ', yam_file_path)
    
    return final_dict_template

In [12]:
#window paramerter defines how many combinations (Reference, A, B) will the participant see
window = 20
start  = 0
ranges = []
while start < len(final_list):
    if start + window > len(final_list):
        ranges.append([start, len(final_list)])
        start+=window
    else:
        ranges.append([start, start+ window])
        start +=window
    

In [13]:
for start, end in ranges:
    out = make_yaml_file(combinations=final_list[start:end], idx_meta=f'{start}_{end}')


    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/idx0_20.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/idx20_40.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/idx40_60.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/idx60_80.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/webmushra/configs/idx80_100.yaml

    20 pages will be created for subjective evalutation
    
Created yaml file at:  /Users/ashishalex/Documents/research/web_drone_project/